<a href="https://colab.research.google.com/github/bjdzliu/ai_lab/blob/main/langchain/PromptTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 两种Prompt模板封装的方式
- PromptTemplate
- ChatPromptTemplate

In [ ]:
!pip3 --quiet install langchain  langchain-openai

In [19]:
from google.colab import userdata
apikey=userdata.get('OPENAI_API_KEY')
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [5]:
llm = OpenAI(temperature=0.9,openai_api_key=apikey)  # default model_name="gpt-3.5-turbo-instruct"
#check the llm object parameters
#print(llm)

## Simple without PromptTemplate



In [ ]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm.invoke(text))

## Directly put template in PromptTemplate.
### Instantiation using from_template (recommended)

In [15]:
template = PromptTemplate.from_template("tell me a joke about  {subject} ")
print("===Template===")
print(template)
print("===Prompt===")
print(template.format(subject='sun'))


===Template===
input_variables=['subject'] template='tell me a joke about  {subject} '
===Prompt===
tell me a joke about  sun 


In [ ]:
simple_prompt=template.format(subject='sun')
ret = llm.invoke(simple_prompt)
print(ret)

## Parameters with PromptTemplate
###  Instantiation using initializer

In [17]:
template = """
I want you to act as a naming consultant for new companies.
What is a good name for a company that makes {product} ?
Where can customer buy these in {country_name}?
"""
prompt_single = PromptTemplate(
    input_variables=["product","country_name"],
    template=template,
)

prompt_str=prompt_single.format(product="colorful socks",country_name="china")

In [ ]:
llm.invoke(prompt_str)

## multiple parameters with PromptTemplate and chain






In [20]:

prompt_template = "Tell me a {parameter_1} and {parameter_2} joke"

#Init a prompt object
prompt = PromptTemplate(
    input_variables=["parameter_1","parameter_2"], template=prompt_template
)

In [21]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [22]:
llm_chain.invoke({"parameter_1":"chinese","parameter_2":"fashion"})

{'parameter_1': 'chinese',
 'parameter_2': 'fashion',
 'text': '\n\nWhy was the Chinese fashion designer always so successful?\n\nBecause he knew how to make a statement with his seams.'}

###ChatOpenAI

In [ ]:
import langchain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-4",openai_api_key=apikey) # 默认是gpt-3.5-turbo
response = llm.invoke("你是谁")
print(response.content)

In [ ]:
### Simple example without parameters
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
llm(messages)


### 对话 ChatPromptTemplate example 1
可以基于历史做回答

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate,AIMessagePromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(openai_api_key=apikey)

In [ ]:
## Example with parameters
template = ChatPromptTemplate.from_messages(
    [
     SystemMessagePromptTemplate.from_template("这里是家用机器人展会，你在一家大型公司的展台，你是产品设计经理,你叫马克"),
     HumanMessagePromptTemplate.from_template("我发现你非常熟悉{product},我很感兴趣"),
     AIMessagePromptTemplate.from_template("很高兴认识你，我是很专业的产品专家"),
     HumanMessagePromptTemplate.from_template("我怎么称呼你,{query}"),
    ]
)


prompt=template.format_messages(
    product="打扫、做饭一体机器人",
    name="马克",
    query="你可以给我介绍一下吗"
)



llm.invoke(prompt)

### 对话 ChatPromptTemplate example 2

In [ ]:
template_2 = ChatPromptTemplate.from_messages([
                    ("system", "You are a my workmate. Your name is {name}."),
                    ("human", "Hello, {name}, how are you?"),
                    ("ai", "I'm doing well, thanks! I am going to the park, If you have anything, you better ask asap"),
                    ("human", "That's good to hear,{query}"),
])

In [ ]:
prompt_2=template_2.format_messages(
    name="jack",
    query="Have you done your work?"
)
llm.invoke(prompt_2)

### Save and Load prompt example
保存格式 JSON


In [ ]:
# init a simple template
single_input_prompt = PromptTemplate(input_variables = ['product'],
                                template = 'What is a good name for a company that makes {product}?, Directly tell me name')

In [ ]:
print("typle is",type(single_input_prompt),"\n","content is: ",single_input_prompt)

typle is <class 'langchain_core.prompts.prompt.PromptTemplate'> 
 content is:  input_variables=['product'] template='What is a good name for a company that makes {product}?, Directly tell me name'


In [ ]:
single_input_prompt.save("Saved_prompt.json")

In [ ]:
with open("Saved_prompt.json") as file:
  file_content = file.read()
  print(  file_content )

{
    "name": null,
    "input_variables": [
        "product"
    ],
    "input_types": {},
    "output_parser": null,
    "partial_variables": {},
    "metadata": null,
    "tags": null,
    "template": "What is a good name for a company that makes {product}?, Directly tell me name",
    "template_format": "f-string",
    "validate_template": false,
    "_type": "prompt"
}


In [ ]:
from langchain.prompts import load_prompt

loaded_prompt = load_prompt("Saved_prompt.json")

In [ ]:
## Method-1 format tempalte with format() is:
loaded_prompt.format(product='colorful socks')

'What is a good name for a company that makes colorful socks?'

In [ ]:
## Method-2 format tempalte with partial() is:
kv={"product":'smart house robot'}
final_prompt=loaded_prompt.partial(**kv)

print(final_prompt.format())


What is a good name for a company that makes smart house robot?, Directly tell me name


In [ ]:
llm.invoke(final_prompt.format())

'\n\n"SmartBots Inc."'